In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
import numpy as np
import requests
import re
#from helpers.integration_helpers import canoicalize_smiles

In [2]:
def get_compound_with_element(data, element):
    return data[data['smiles'].str.contains(element)]

def get_smiles(compound):
    #print(compound)
    smilesUrl = "https://opsin.ch.cam.ac.uk/opsin/"+compound+".smi"
    requestSmiles = requests.get(smilesUrl)
    if requestSmiles.status_code == 400 or requestSmiles.status_code == 404:
        return None
    else:
        m = Chem.MolFromSmiles(requestSmiles.text)
        if m != None:
            return Chem.MolToSmiles(Chem.MolFromSmiles(requestSmiles.text)) 
        else:
            return None
    
def celsiusToKelvin(temp):
    return temp + 273.15

def get_compound_with_element(data, element):
    return data[data['smiles'].str.contains(element)]

In [3]:
integrated_data = pd.read_csv('../data/integrated_dataset.csv')
print(integrated_data['source'].unique())
print(integrated_data['smiles'].unique().shape)
#integrated_data[integrated_data['smiles'].str.contains('Si', regex=False)]['smiles'].unique().shape
#integrated_data.head()
#integrated_data.drop(columns=['Unnamed: 0'], inplace=True)
#list(integrated_data)
#integrated_data.to_csv('../data/integrated_dataset.csv', index=False)

['pubchem' 'carroll11' 'chen14' 'Saldana11' 'gelest_germanium'
 'gelest_metal-organics' 'pan12' 'gelest_silanes' 'gelest_tin'
 'hazardous materials' 'le15' 'didier14' 'pan07' 'caroll15' 'didier12'
 'godinho11' 'katritzky07' 'carroll10' 'wang11' 'patel10']
(10630,)


## Carroll

In [ ]:
carroll = pd.read_csv('../data/Carroll11.csv', encoding = "ISO-8859-1")

In [ ]:
print(list(carroll))
#carroll = data[['Compound', 'TFP(K)Ref.']]
#carroll['source'] = 'carroll11'
#carroll['pure substance'] = 1
carroll.head()

## Saldana

In [ ]:
fuels = pd.read_csv('../data/fuels.csv', encoding = "ISO-8859-1")
fuels.head()

## Pan12

In [ ]:
pan12 = pd.read_csv('../data/pan12.csv', encoding = "ISO-8859-1")
pan12.head()

## chen14

In [ ]:
chen14 = pd.read_csv('../data/chen14.csv', encoding = "ISO-8859-1")

In [ ]:
chen14.head()

In [ ]:
# canonicalize carroll smiles
for i in range(0, len(carroll)):
    carroll.iloc[i]['smiles'] = Chem.MolToSmiles(Chem.MolFromSmiles(carroll.iloc[i]['smiles']))


In [ ]:
#canonicalize smiles with rdkit
result['canonical smiles'] = None
for idx, row in result.iterrows():
    result.loc[idx,'canonical smiles'] = Chem.MolToSmiles(Chem.MolFromSmiles(result.iloc[idx]['smiles']))

In [ ]:
for idx, row in result.iterrows():
    result.loc[idx,'canonical smiles'] = Chem.MolToSmiles(Chem.MolFromSmiles(result.loc[idx]['smiles']))

## Add Gelest to dataset

In [ ]:
silanes = pd.read_csv('../data/silanes_all.csv')
metal_organic = pd.read_csv('../data/metal-organics_all.csv')
tin = pd.read_csv('../data/tin_all.csv')
germanium  = pd.read_csv('../data/germanium_all.csv')
print(list(silanes))
print(list(metal_organic))
print(list(tin))
print(list(germanium))

In [ ]:
#result = result[['compound', 'smiles', 'flashpoint', 'pure substance', 'source']]
silanes_pure = silanes[silanes['pure substance'] == 1]
metal_organic_pure = metal_organic[metal_organic['pure substance'] == 1]
tin_pure = tin[tin['pure substance'] == 1]
germanium_pure = germanium[germanium['pure substance'] == 1]

## pubchem integration

In [ ]:
pubchem = pd.read_csv('../data/pubchem_data.csv')
pubchem = pubchem[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
germanium_pure = germanium_pure[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
tin_pure = tin_pure[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
metal_organic_pure = metal_organic_pure[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
silanes_pure = silanes_pure[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
hazardous_mat = hazardous_mat[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]


In [ ]:
print(list(pubchem))
print(list(carroll))
print(list(chen14))
print(list(fuels))
print(list(germanium_pure))
print(list(metal_organic_pure))
print(list(pan12))
print(list(silanes_pure))
print(list(tin_pure))
print(list(hazardous_mat))

In [ ]:
def canoicalize_smiles(data):
    for idx, row in data.iterrows():
        m = Chem.MolFromSmiles(data.iloc[idx]['smiles'])
        if m != None:
            data.iloc[idx]['smiles'] = Chem.MolToSmiles(m)
        else:
            data.iloc[idx]['smiles'] = None
    return data

In [ ]:
frames = [pubchem, carroll, chen14, fuels, germanium_pure, metal_organic_pure, pan12, silanes_pure, tin_pure, hazardous_mat]
result = pd.concat(frames)

In [ ]:
canonical_smiles_data = canoicalize_smiles(result)

In [ ]:
print(canonical_smiles_data.shape)
print(canonical_smiles_data.dropna().shape)
canonical_smiles_data.dropna(inplace=True)

In [ ]:
canonical_smiles_data.to_csv('../data/integrated_dataset.csv', index=False)

In [ ]:
def remove_duplicates(data):
    duplicates = data[data.duplicated(subset='smiles')]
    result = data.drop_duplicates(subset='smiles', keep=False)#[~duplicates]
    #for each unique smiles that has duplicates
    for smiles in data[data.duplicated(subset='smiles')]['smiles'].unique():
        dup_rows = data.loc[data['smiles'] == smiles]
        if dup_rows['flashpoint'].unique().shape[0] == 1:
            # remove all but one
            result = result.append(dup_rows.iloc[0], sort=False)
        else:
            if dup_rows['flashpoint'].std() < 5:
                # add 1 back
                result = result.append(dup_rows.iloc[0], sort=False)
    return result  
            

In [ ]:
pure_compounds = data.loc[data['pure substance'] == 1]
pure_compounds.shape

In [ ]:
data[data.duplicated(subset='smiles')].shape

In [ ]:
data = data.loc[data['pure substance'] == 1]
print(data.shape)
no_dups = remove_duplicates(data)
print(no_dups.shape)

In [ ]:
no_dups[no_dups.duplicated(subset='smiles')]
#no_dups.loc[no_dups['smiles'] == 'CC1=CCCC(=C)C2CC(C2CC1)(C)C']
no_dups.to_csv('../data/integrated_dataset.csv', index=False)

In [ ]:
result = data.drop_duplicates(subset='smiles', keep=False)
result.shape

In [ ]:
data[data.duplicated(subset='smiles')]['smiles'].unique()

### Hazardous Materials

In [ ]:
data = pd.read_csv('../data/hazardous_materials.csv')
haz = data[['Common name', 'Data type', 'Measurement name', 'Measurement value', 'Measurement units']]

In [ ]:
flashpoints = haz.loc[haz['Measurement name'] == 'Flash point']

In [ ]:
print(list(flashpoints))

In [ ]:
flashpoints['smiles'] = flashpoints['Common name'].apply(get_smiles)

In [ ]:
flashpoints = flashpoints[~flashpoints['Measurement value'].str.contains('<')]
flashpoints = flashpoints[~flashpoints['Measurement value'].str.contains('>')]
flashpoints = flashpoints[~flashpoints['Measurement value'].str.contains('~')]
pattern = '\\w+-\\w'
flashpoints['Measurement value'] = flashpoints['Measurement value'].str.replace(pattern, 'none', regex=True)
#flashpoints = flashpoints[~flashpoints['Measurement value'].str.contains('none')]
flashpoints['Measurement value'] = pd.to_numeric(flashpoints['Measurement value'], errors='coerce')

flashpoints.head()

In [ ]:
flashpoints['flashpoint'] = flashpoints['Measurement value'].apply(celsiusToKelvin)
flashpoints.drop(columns=['Measurement name','Data type', 'Measurement units', 'Measurement value'], inplace=True)
#flashpoints.rename(columns={'Common name' : 'compound', 'Measurement value' : 'flashpoint'}, inplace=True)

In [ ]:
flashpoints.dropna(inplace=True)
flashpoints.rename(columns={'Common name' : 'compound'}, inplace=True)
#flashpoints.drop(df.columns[i], axis=1)
flashpoints['pure substance'] = 1
flashpoints['source'] = 'hazardous materials'
hazardous_mat = flashpoints[['compound', 'flashpoint', 'pure substance', 'smiles', 'source']]
hazardous_mat.shape

In [ ]:
hazardous_mat.head()

In [ ]:
#data = pd.read_csv('../data/integrated_dataset.csv')
#frames = [data, hazardous_mat]
#result = pd.concat(frames)
no_dups = remove_duplicates(result)
print(result.shape)
print(no_dups.shape)
#no_dups.to_csv('../data/integrated_dataset.csv', index=False)

## Le 2015

In [ ]:
og_le15 = pd.read_csv('../data/le15.csv')
og_le15_2 = pd.read_csv('../data/le15_2.csv')
#le15.rename(columns={"MolID" : "compound", "FP" : "flashpoint"}, inplace=True)
#print(le15.head())
#print(le15_2.head())

In [ ]:
le15['flashpoint'] = le15['flashpoint'].apply(celsiusToKelvin)
le15_2['flashpoint'] = le15_2['flashpoint'].apply(celsiusToKelvin)

In [ ]:
le15['smiles'] = le15['compound'].apply(get_smiles)
le15_2['smiles'] = le15_2['compound'].apply(get_smiles)

In [ ]:
le15['flashpoint'] = og_le15['flashpoint'].apply(celsiusToKelvin)
le15_2['flashpoint'] = og_le15_2['flashpoint'].apply(celsiusToKelvin)
le15_2.head()

In [ ]:
le15_smiles = le15[le15.smiles.isnull() == False]
le15_2_smiles = le15_2[le15_2.smiles.isnull() == False]
le15_no_smiles = le15[le15.smiles.isnull() == True]
le15_2_no_smiles = le15_2[le15_2.smiles.isnull() == True]

In [ ]:
def clean_string(string):
    #remove ending number
    if ',' in string:
        return string
    pat = r'\w+\D'
    prog = re.compile(pat)
    match = prog.match(string)
    if match != None:
        string = match.group(0)
    #remove [0-9]{1,2}oC
    string = re.sub(r'[0-9]{1,2}oC', '', string)
    #add , to [0-9]{2}
    pat2 = r'^[0-9]{2}'
    if re.search(pat2, string) != None:
        string = string[0] + ',' + string[1:len(string)]
    return string
    #else:
    #    return string

In [ ]:
print(le15_no_smiles.shape)
le15_no_smiles['cleaned_compound'] = ''
le15_no_smiles['cleaned_compound'] = le15_no_smiles['compound'].apply(clean_string)
le15_no_smiles['smiles'] = le15_no_smiles['cleaned_compound'].apply(get_smiles)

In [ ]:
print(le15_2_no_smiles.shape)
le15_2_no_smiles['cleaned_compound'] = ''
le15_2_no_smiles['cleaned_compound'] = le15_2_no_smiles['compound'].apply(clean_string)
le15_2_no_smiles['smiles'] = le15_2_no_smiles['cleaned_compound'].apply(get_smiles)

In [ ]:
le15_2_no_smiles.dropna(inplace=True)
le15_no_smiles.dropna(inplace=True)
frames = [le15_2_no_smiles, le15_no_smiles]
le_reclaimed = pd.concat(frames)
print(le_reclaimed.shape)

In [ ]:
le_reclaimed.drop(columns=['compound'], inplace=True)
le_reclaimed.rename(columns={'cleaned_compound' : 'compound'}, inplace=True)
le_reclaimed['source'] = 'le15'
le_reclaimed['pure substance'] = 1
le_reclaimed = le_reclaimed[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]

In [ ]:
le_reclaimed#.head()
all_data[all_data['smiles'] == 'Cl[Si](Cl)(Cl)C(Br)CBr']

In [ ]:
all_data = pd.read_csv('../data/integrated_dataset.csv')
print(all_data['smiles'].unique().shape)

In [ ]:
#frames = [all_data, le_reclaimed]
#result = pd.concat(frames)
result.smiles.unique().shape
#result.to_csv('../data/integrated_dataset.csv', index=False)

In [ ]:
le15_smiles['source'] = 'le15'
le15_smiles['pure substance'] = 1
le15_smiles = le15_smiles[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
le15_smiles['flashpoint'] = le15_smiles['flashpoint'].apply(celsiusToKelvin) 
print(list(le15_smiles))

In [ ]:
frames = [all_data, le15_smiles]
result = pd.concat(frames)
result.smiles.unique().shape

In [ ]:
result['source'].unique()

In [ ]:
test = result.drop_duplicates(subset='smiles', keep='first')
test.shape
test = test[['smiles', 'flashpoint']]
test.to_csv('all_fp.csv', index=False)

In [ ]:
#result[result['source'] == 'le15'].head()
result = pd.read_csv('../data/integrated_dataset.csv')

### Wang11

In [ ]:
integrated_data[integrated_data['compound'] == 'Methyltrichlorosilane']

In [ ]:
wang = pd.read_csv('../data/wang11.csv')
wang.head()
print(wang.shape)

In [ ]:
wang['smiles'] = wang['compound'].apply(get_smiles)
wang.head()

In [ ]:
wang[wang['smiles'].isnull() == False].shape

In [ ]:
wang.dropna(inplace=True)
wang.shape

In [ ]:
wang['pure substance'] = 1
wang['source'] = 'wang11'
print(list(integrated_data))
print(list(wang))

In [ ]:
wang = wang[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]

In [ ]:
integrated_data = integrated_data[integrated_data.source != 'wang11']
frames = [integrated_data, wang]
integrated_data = pd.concat(frames)
integrated_data['smiles'].unique().shape

In [ ]:
integrated_data.to_csv('../data/integrated_dataset.csv')

### Didier14

In [ ]:
didier14 = pd.read_csv('../data/didier14.csv')
didier14.dropna(inplace=True)
print(didier14.shape)
didier14['smiles'] = didier14['Name'].apply(get_smiles)
didier14['source'] = 'didier14'
didier14['pure substance'] = 1
print(list(didier14))

In [ ]:
didier14.dropna(inplace=True)
print(didier14.shape)

In [ ]:
didier14.rename(index=str, columns={'Name' : 'compound', 'exp.' : 'flashpoint'}, inplace=True)
didier14.drop(columns=['CASRN', 'calc.'])
didier14 = didier14[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]

In [ ]:
frames = [integrated_data, didier14]
integrated_data = pd.concat(frames)
integrated_data['smiles'].unique().shape

In [ ]:
smi = Chem.MolToSmiles(Chem.MolFromSmiles('ClC(C)CCC(C)C'))
#result[result['smiles'] == smi]

In [ ]:
combined[combined['smiles'] == smi]

### Yong_Pan07

In [ ]:
pan07 = pd.read_csv('../data/yong_pan07.csv')
print(pan07.shape)
print(list(pan07))

In [ ]:
pan07['smiles'] = pan07['Compound'].apply(get_smiles)
pan07.dropna(inplace=True)
print(pan07.shape)
pan07['source'] = 'pan07'
pan07['pure substance'] = 1
print(list(pan07))
print(list(integrated_data))

In [ ]:
pan07.rename(index=str, columns={'Compound' : 'compound', 'Experimental FP (K)' : 'flashpoint'}, inplace=True)
pan07.drop(columns=['No.', 'Predicted FP (K)', 'Deviation (K)'])
pan07 = pan07[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]

In [ ]:
frames = [integrated_data, pan07]
integrated_data = pd.concat(frames)
integrated_data['smiles'].unique().shape

### carroll15

In [ ]:
carroll15 = pd.read_csv('../data/carroll15.csv')
print(carroll15.shape)
print(list(carroll15))

In [ ]:
carroll15['smiles'] = carroll15['Compound'].apply(get_smiles)
carroll15.dropna(inplace=True)
print(carroll15.shape)
carroll15['source'] = 'caroll15'
carroll15['pure substance'] = 1
print(list(carroll15))
print(list(integrated_data))

In [ ]:
carroll15.rename(index=str, columns={'Compound' : 'compound', 'T FP (K) Lit.' : 'flashpoint'}, inplace=True)
carroll15.drop(columns=['Entry', 'Y BP Pred.'])
carroll15 = carroll15[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]

In [ ]:
frames = [integrated_data, carroll15]
integrated_data = pd.concat(frames)
integrated_data['smiles'].unique().shape

### Didier12

In [ ]:
didier12 = pd.read_csv('../data/didier12.csv')
didier12 = didier12[['Molecule', 'Exp.(K)']]
print(didier12.shape)
print(list(didier12))

In [ ]:
didier12['smiles'] = didier12['Molecule'].apply(get_smiles)
didier12.dropna(inplace=True)
print(didier12.shape)
didier12['source'] = 'didier12'
didier12['pure substance'] = 1
print(list(didier12))
print(list(integrated_data))

In [ ]:
didier12.rename(index=str, columns={'Molecule' : 'compound', 'Exp.(K)' : 'flashpoint'}, inplace=True)
didier12 = didier12[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]

In [ ]:
frames = [integrated_data, didier12]
integrated_data = pd.concat(frames)
integrated_data['smiles'].unique().shape

In [ ]:
integrated_data.to_csv('../data/integrated_dataset.csv', index=False)

In [ ]:
def remove_invalid_smiles(data):
    invalid = []
    for index, row in data.iterrows():
        #print(row['smiles'])
        if Chem.MolFromSmiles(row['smiles']) == None:
            invalid.append(row['smiles'])
            #data.drop(row.index[0], inplace=True)
    print('invalid smiles strings')
    print('------------------------------')
    print(invalid)
    print('------------------------------')
    for smi in invalid:
        data.drop([data[data['smiles'] == smi].index[0]], inplace=True)

In [ ]:
check_valid_smiles(integrated_data)

In [ ]:
integrated_data.to_csv('../data/integrated_dataset.csv', index=False)

In [ ]:
integrated_data['smiles'].unique()

## Godinho11

In [ ]:
#godinho = pd.read_csv('../data/godinho11.csv')
#integrated_dataset = pd.read_csv('../data/integrated_dataset.csv')
integrated_dataset['smiles'].unique().shape
list(godinho)

In [ ]:
godinho['smiles'] = godinho['Compound'].apply(get_smiles)

In [ ]:
godinho['source'] = 'godinho11'
godinho['pure substance'] = 1
godinho.rename(index=str, columns={'Compound' : 'compound', 'T FP (K)' : 'flashpoint'}, inplace=True)
godinho = godinho[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
list(godinho)

In [ ]:
godinho.head()

In [ ]:
print(godinho.shape)
godinho.dropna(inplace=True)
print(godinho.shape)
frames = [integrated_dataset, godinho]
integrated_dataset = pd.concat(frames)
integrated_dataset['smiles'].unique().shape

In [ ]:
integrated_dataset.to_csv('../data/integrated_dataset.csv')
#integrated_dataset[integrated_dataset['smiles'].str.contains('Si', regex=False)]
integrated_dataset.isnull().sum()

In [ ]:
integrated_dataset.drop(columns=['Compound','T FP (K)'], inplace=True)

In [ ]:
integrated_dataset[integrated_dataset['source'] == np.nan]

In [ ]:
integrated_dataset.dropna(inplace=True)#['smiles'].unique().shape

In [ ]:
integrated_dataset['smiles'].unique().shape

### Patel10

In [6]:
patel10 = pd.read_csv('../data/patel10_smiles.csv')
print(patel10.shape)
#patel10['smiles'] = patel10['compound'].apply(get_smiles)
patel10.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'],inplace=True)
#print(patel10.shape)
patel10.head()

(235, 6)


compound  flashpoint   smiles
0             methanol     283.706       CO
1              ethanol     286.483      CCO
2          propan-1-ol     298.150     CCCO
3           butan-2-ol     295.372  CCC(C)O
4  2-methylpropan-1-ol     303.150  CC(C)CO

In [22]:
patel10.dropna().head()#isnull().any().sum()


compound  Tf(exptl) (K)   smiles
0             methanol        283.706       CO
1              ethanol        286.483      CCO
2          propan-1-ol        298.150     CCCO
3           butan-2-ol        295.372  CCC(C)O
4  2-methylpropan-1-ol        303.150  CC(C)CO

In [7]:
#patel10.rename(columns={'Tf(exptl) (K)' : 'flashpoint'},inplace=True)
patel10['source'] = 'patel10'
patel10['pure substance'] = 1
patel10 = patel10[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]

In [9]:
#integrated_data = integrated_data[integrated_data['source'] != 'patel10']
patel10.head()

smiles             compound  flashpoint   source  pure substance
0       CO             methanol     283.706  patel10               1
1      CCO              ethanol     286.483  patel10               1
2     CCCO          propan-1-ol     298.150  patel10               1
3  CCC(C)O           butan-2-ol     295.372  patel10               1
4  CC(C)CO  2-methylpropan-1-ol     303.150  patel10               1

In [10]:
frames = [integrated_data, patel10]
integrated_data = pd.concat(frames)
print(integrated_data['smiles'].unique().shape)
integrated_data.to_csv('../data/integrated_dataset.csv', index=False)

(10630,)


C:\Users\Nathaniel\Miniconda3\envs\moleprop\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


### Katritzky07

In [ ]:
katritzky07 = pd.read_csv('../data/Katritzky07.csv')
print(katritzky07.shape)
katritzky07['smiles'] = katritzky07['compound'].apply(get_smiles)
katritzky07.dropna(inplace=True)
print(katritzky07.shape)
katritzky07.head()

In [ ]:
katritzky07.rename(columns={'TFP (exp)' : 'flashpoint'},inplace=True)
katritzky07['source'] = 'katritzky07'
katritzky07['pure substance'] = 1
katritzky07 = katritzky07[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]

In [ ]:
frames = [integrated_data, katritzky07]
integrated_data = pd.concat(frames)
print(integrated_data['smiles'].unique().shape)

### carroll10

In [ ]:
carroll10 = pd.read_csv('../data/carroll10.csv')
print(carroll10.shape)
carroll10['smiles'] = carroll10['Compound'].apply(get_smiles)
carroll10.dropna(inplace=True)
print(carroll10.shape)
carroll10.head()

In [ ]:
carroll10.rename(columns={'Compound' : 'compound', 'TFP (K)' : 'flashpoint'},inplace=True)
carroll10['source'] = 'carroll10'
carroll10['pure substance'] = 1
carroll10 = carroll10[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
carroll10.head()

In [ ]:
frames = [integrated_data, carroll10]
integrated_data = pd.concat(frames)
print(integrated_data['smiles'].unique().shape)

In [ ]:
integrated_data.to_csv('../data/integrated_dataset.csv', index=False)

### Check for duplicates in source

In [ ]:
re.compile('\\d+-\\d', re.ASCII)
value = '2–4'
print(re.search('\\d+-\\d', value, re.ASCII))
#hch
#frames = [hch, integrated_data]
#result = pd.concat(frames)

In [ ]:
result['smiles'].unique().shape

In [ ]:
integrated_data.isnull().sum()

In [15]:
integrated_data[integrated_data['source'] == 'le15'][integrated_data[integrated_data['source'] == 'le15'].duplicated()]

compound  flashpoint  pure substance  \
4515                   (E)-2-methylbut-2-enal      291.45             1.0   
5403              1,2,4-trimethylcyclopentane      282.35             1.0   
5410              1,2,4-trimethylcyclopentane      282.35             1.0   
6031              1-ethyl-2-methylcyclohexane      305.05             1.0   
6747   1,3,3a,4,5,6,7,7a-octahydroinden-2-one      352.75             1.0   
6881   8-methyl-8-azabicyclo[3.2.1]octan-3-ol      385.65             1.0   
7282                       pentan-2-ylbenzene      333.75             1.0   
13753                           Cyclohexanone      317.15             1.0   
13965                          Pentanenitrile      313.75             1.0   
14078                           1,2propadiene      172.00             1.0   
14079                               Aziridine      262.05             1.0   
14080                            Acetaldehyde      246.15             1.0   
14081                           Dimethylether      232.15             1.0   
14082                                 ethanol      286.15             1.0   
14083                         2propenenitrile      273.15             1.0   
14084                            1,3butadiene      197.00             1.0   
14085                                1butyne-      210.15             1.0   
14086                                2butyne-      224.15             1.0   
14087                               2propenal      254.25             1.0   
14088                        MethylIsocyanate      255.15             1.0   
14089                       2methylaziridine-      263.15             1.0   
14090                              Allylamine      244.15             1.0   
14091                                 Acetone      253.15             1.0   
14092                            Allylalcohol      294.15             1.0   
14093                         Propionaldehyde      243.15             1.0   
14094                          Isopropylamine      256.15             1.0   
14095                              Aceticacid      312.15             1.0   
14096                     hydroxyacetaldehyde      360.15             1.0   
14097                           Methylformate      254.15             1.0   
14098                               2propanol      285.15             1.0   
...                                       ...         ...             ...   
14803                   VinylTriacetoxySilane      361.15             1.0   
14804                       1,3dibromoBenzene      366.15             1.0   
14805                   OctamethylTrisiloxane      303.15             1.0   
14806                PhenethylTrichloroSilane      364.15             1.0   
14807                   PhenylTriethoxySilane      369.15             1.0   
14808                             Heptadecane      421.15             1.0   
14809            3chloropropylTriethoxySilane      335.15             1.0   
14810              2bromoethyltrichlorosilane      313.15             1.0   
14811            3bromopropylTrimethoxySilane      355.15             1.0   
14812                        Diallylphthalate      438.75             1.0   
14813                     HexaethylDisiloxane      349.15             1.0   
14814                 Bromopentafluorobenzene      360.15             1.0   
14815     3methacryloxyPropylTrimethoxySilane      365.15             1.0   
14816                              Octadecane      438.15             1.0   
14817             3bromopropyltrichlorosilane      361.15             1.0   
14818         TrisTrimethylSiloxyMethylSilane      336.15             1.0   
14819                              Nonadecane      439.15             1.0   
14821                             StearicAcid      469.00             1.0   
14822            OctamethylCyclotetrasilazane      339.15             1.0   
14823            OctamethylCyclotetrasiloxane      330.15             1.0   
14824          PropylTrisdimethyl

In [17]:
integrated_data[integrated_data['source'] == 'le15']['smiles'].unique().shape

(7597,)

In [27]:
def remove_duplicates_by_source(dataframe):
    sources = dataframe.source.unique()
    for source in sources:
        sourceframe = dataframe[dataframe['source'] == source]
        dataframe = dataframe[dataframe['source'] != source]
        sourceframe.drop_duplicates(subset ='smiles',keep='first', inplace=True)
        frames = [dataframe, sourceframe]
        dataframe = pd.concat(frames)
    return dataframe

def check_null_values(dataframe):
    return dataframe.isnull().sum()

In [28]:
copy = integrated_data
change = remove_duplicates_by_source(copy)
print(integrated_data.shape)
print(change.shape)

C:\Users\Nathaniel\Miniconda3\envs\moleprop\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


(16386, 5)
(14692, 5)


In [35]:
change[change['smiles'] == 'CCCCO.CCCCO.CCCCO.CCCCO.[Ti]']
#copy[copy.source == 'pubchem'][copy[copy.source == 'pubchem'].duplicated()]

compound  flashpoint  pure substance  \
1521  butan-1-ol;titanium      350.15             1.0   

                            smiles   source  
1521  CCCCO.CCCCO.CCCCO.CCCCO.[Ti]  pubchem

In [15]:
pubchem[pubchem['smiles'].duplicated()]
pubchem[pubchem['smiles'] == 'CC1(C2CCC1(C(C2)O)C)C']
pubchem.drop_duplicates(subset ='smiles',keep='first').shape

(1612, 5)

In [16]:
copy[copy['smiles'] == 'CCCCO.CCCCO.CCCCO.CCCCO.[Ti]']


(1755, 5)

In [39]:
change[change['compound'] == 'heptanes']

compound  flashpoint  pure substance smiles   source
16238  heptanes      272.15             1.0    NaN  patel10

In [44]:
change.dropna(inplace=True)
change.isnull().sum()

compound          0
flashpoint        0
pure substance    0
smiles            0
source            0
dtype: int64

In [45]:
change.to_csv('../data/integrated_dataset.csv', index=False)